# How to compute averages of 

This is a tutorial on how to compute statistical average of several quatenrions and Rotation matrices. Demonstrating example of this can be shown with quaternions. 

### Quaternion Averaging

This tutorial is influenced by the followin paper: http://www.acsu.buffalo.edu/~johnc/ave_quat07.pdf 
And describes the algorithm used there. As a demonstration, imagine that you have some vector of quatenrions and you want to find the average. It is tempting to find average with the following method:

In [ ]:
import numpy as np
import numpy.linalg as LA
def average_q_bad(vector_of_quaternions):
    q_average = np.sum(vector_of_quaternions, axis=0)/ len(vector_of_quaternions)
    return q_average/LA.norm(q_average)

However, this equation has two issues:
1. Quaternion shouold be normalized, because average does not always produce valid quaternion
2. remember that $q$ and $-q$ correspond to the same rotation. but naive average leads to bad results. Here is example:

In [ ]:
q_mean = np.array([1, 0, 0, 0])

# making several same rotations by changing signs even number of times:
Q_alternating_signs = np.array([(-1)**i * q_mean for i in range(10)])

In [ ]:
average_q_bad(Q_alternating_signs)

C:\Users\1\AppData\Local\Temp\ipykernel_6444\3288014740.py:5: RuntimeWarning: invalid value encountered in divide
  return q_average/LA.norm(q_average)


array([nan, nan, nan, nan])

As you can see, something bad happened, because we divided to zero. You can say that this type of vector never really happens, so lets demonstrate another example:

In [ ]:
q_mean = np.array([1, 0, 1, 0])
q_mean = q_mean/LA.norm(q_mean)
q_mean

array([0.70710678, 0.        , 0.70710678, 0.        ])

In [56]:
Q_random_noise = np.array([(-1)**i *q_mean + 0.1*(2*np.random.rand(4) - 1) for i in range(100)])

In [57]:
average_q_bad(Q_random_noise)

array([ 0.03353744, -0.54693355,  0.15518062,  0.82198413])

Now, you not only don't get the quaternion you wanted, but now you have completely wrong quaterions!

#### Correct Quaternion average
One of the things about quaternions, they should satisfy the following condition: $q^Tq =1$ 
The way we will solve the quaternion average is by computing following mean: 
$ \sum^N_{i=1} || R(q) - R(q_i) ||_F$ , here $R(q)$- quaternion to rotation matrix transform and $|| \cdot ||_F$ is Frobenius norm. It is possible to show that the minimizing for $q$ leads to maximizing followin equaition:

$ 
\begin{align}
max_q q^T M q 
\end{align}
$

To satisfy the constrain on length, we have to solve the following problem: 
$
\begin{align}
max_\lambda min_q - q^T M q + \lambda(q^Tq - 1)
\end{align}
$

Therefore, if we find the derfivative  with $\lambda$ we have our constraint and if we have derivative with $q$ we need to have zero gradient for optimality, because $M$ is positive definite: 
$
\begin{align}
(M - \lambda I) q = 0
\end{align}
$
You should recognize this equation, this is basically finding eigenvalues and eigenvectors. Keep in mind, we need to maximize $\lambda$, therefore, we are interested in largest eigenalue and corresponding eigenvectgor.

Algorithm for quaternions becomes the following:

In [62]:
def average_q_correct(vector_of_quaternions):
    # create transpose multiplication
    M = vector_of_quaternions.transpose() @ vector_of_quaternions
    # find the most dominant eigenvector and eigenvalues
    u , s, v = LA.svd(M)
    #index of maximum eigenvalue
    i = np.argmax(s)
    #return eigenvector corrsponding to maximum eigenvalue
    return u[:, i]

In [59]:
average_q_correct(Q_alternating_signs)

array([1., 0., 0., 0.])

In [61]:
average_q_correct(Q_random_noise)

array([-0.69870561, -0.00186204, -0.71540433,  0.0019091 ])

As you can see, the above solution actually finds the correct quaternion averages. 

## Rotation matrix averages

What if instead of quaternions we had rotation matrices? You could find the quaternions and use the above equation or you can try another way of solving the equation. For this, we can use the following property:

$ min_q \sum^N_{i=1} || R(q) - R(q_i) ||_F = min_q || R(q) - \sum^N_{i=1}R(q_i) ||_F$

Derivation that this is eqivalent problem is left as exercise. Therefore, you can find the average of the rotation matrices and try to find the bet fit rotation to that average. How to find the best fit rotation matrix to the average? You can use the following paper: https://matthias-research.github.io/pages/publications/stablePolarDecomp.pdf

Here is the algorithm:

In [218]:
from scipy.spatial.transform import Rotation as R
def fit_rotation(matrix_A, guess_matrix, iter):
    for _ in range(iter):
        r = guess_matrix
        # find cross product between A and guess matrix
        cross_v = np.sum([ np.cross(x, y) for x, y in zip(r.transpose(), matrix_A.transpose())], axis=0)
        # dot product between A and guess matrix
        dot_v = np.sum([ np.dot(x, y) for x, y in zip(r.transpose(), matrix_A.transpose())], axis=0)
        # divide cross product to dot product
        omega = cross_v/(abs(dot_v) + 1.e-9)
        #check rotation angle
        w = LA.norm(omega)
        if w< 1e-9:
            #if angle is very small break
            break
        # convert oemga to rotation with exponential map
        q = R.from_rotvec(omega).as_matrix()
        # apply rotation to initial guess
        guess_matrix = q @ guess_matrix
    # return as quaternion
    return R.from_matrix(guess_matrix).as_quat()


In [249]:
A = R.from_quat(q_mean).as_matrix() # matrix from quaternion to fit
v = R.from_quat([1,0,0,0]).as_matrix() # guess
estimate = fit_rotation(matrix_A=A, guess_matrix=v, iter=100)

In [250]:
estimate

array([0.70710678, 0.        , 0.70710678, 0.        ])

In [243]:
q_mean

array([0.70710678, 0.        , 0.70710678, 0.        ])

As you can see, the quaternion estimate is spot on. Now we can do average:

In [246]:
A_s = np.array([A + 0.1*(2*np.random.rand(3,3)-1) for _ in range(10)])

Here is estimation algorithm:

In [247]:
def rotation_estimate(array_of_matrices):
    A_mean = np.sum(array_of_matrices, axis=0)/len(array_of_matrices)
    guess = R.from_quat([1,0,0,0]).as_matrix() # guess
    return fit_rotation(A_mean, guess, 100)

In [248]:
rotation_estimate(A_s)

array([ 0.70927516, -0.00646937,  0.70490137, -0.000981  ])

As you can see, the rotation estimate is very close to what we got before. In some sense, it might be even precise. Of course, the rotation fit algorithm can be implemented efficiently in C++ and imported to python. but here, we use it for demonstration